In [2]:
import xlrd 
import pandas
from pandas import DataFrame

# import data from excel 
if __name__ == '__main__' :
    book = xlrd.open_workbook ('C:/research/python/data/Bric_seq_160119_1.xlsx')
    sheet1 = book.sheet_by_name('PUM1')
    data1 = []
    data1 = [[sheet1.cell_value(r, col)
                for col in range(sheet1.ncols)]
                    for r in range(sheet1.nrows)]

# numbers are registered as numbers, while leaving the strings alone
for i in range (len (data1)):
    for j in range(len(data1[0])):
        try:
            data1[i][j] = float (data1[i][j])
        except:
            pass

# Wrap the data in a pandas DataFrame
df = DataFrame(data1[1:], columns = data1[0])

In [22]:
df.head(3)

,symbol,status,RPKM_siSte,RPKM_siPUM1,log2(fold_change),p_value,q_value,T0_con,T1_ con,T2_cpn,...,half_life_siPUM1,log2_t_half,p_Welch,3UTR_length,PUM_motif,HalfLife_siStealth_PUM1_study,HalfLife_siCTRL_PUM2_study,HalfLife_siCTRL_PUM1_2_study,HalfLife_siPUM1_PUM1_study,p_Grubbs
0,A1BG,NOTEST,0.011698,0.022679,0.955057,1,1,0,0,0,...,NaN,NaN,NaN,216,0,NaN,NaN,NaN,NaN,NaN
1,A1CF,NOTEST,0.048423,0.057783,0.254932,1,1,1,0,0,...,NaN,NaN,NaN,7320,3,NaN,NaN,NaN,NaN,NaN
2,A2M,NOTEST,0.098657,0.004655,-4.40572,1,1,0,0,0,...,NaN,NaN,NaN,115,0,NaN,NaN,NaN,NaN,NaN


In [52]:
import matplotlib.pyplot as plt
import numpy as np

# make a list
Pg_list = list()
P_up = list()

for i in range(1, 21):
    Pg = 0.05 * i
    Pg_list.append(Pg)
    
print Pg_list

[0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001, 1.0]


In [47]:
Pg_up = list()

for Pg in Pg_list:

    # extract the genes of which q_value is lower than q    
    df_p = df[df['p_Grubbs'] <= Pg]
    
    # define the genes which increases (df_up) or decrease (_down) by siPUM 
    df_up = df_p[df_p['log2_t_half'] >0]
    
    # count the number of genes containing PUM motif
    df_up_motif = df_up[df_up['PUM_motif'] >0]
        
    #estimate the motif fraction
    percent_up = 1.0 * df_up_motif['PUM_motif'].count() / df_up['PUM_motif'].count()
    
    Pg_up.append(percent_up)    

print Pg_up
    

[0.30040871934604907, 0.3131046613896218, 0.31680731364275666, 0.31413928248417244]


In [53]:
P_list = list()
Pw_list = list()

for i in range(1, 21):  
    # p_Welch t-test amd p_Grubbs were modified by 0.01
    Pw = 0.05 * i
    Pw_list.append(Pw)

    # extract the genes of which p_value is lower than p_Welch    
    df_Pw = df[df['p_Welch'] <= Pw]
    
    Pg_up = list()

    for Pg in Pg_list:

        # extract the genes of which q_value is lower than q    
        df_p = df_Pw[df_Pw['p_Grubbs'] <= Pg]
    
        # define the genes which increases (df_up) or decrease (_down) by siPUM 
        df_up = df_p[df_p['log2_t_half'] >0]
    
        # count the number of genes containing PUM motif
        df_up_motif = df_up[df_up['PUM_motif'] >0]
        
        #estimate the motif fraction
        percent_up = 1.0 * df_up_motif['PUM_motif'].count() / df_up['PUM_motif'].count()
        Pg_up.append(percent_up)
        
    P_list.append(Pg_up)

print P_list


[[0.3219178082191781, 0.3299492385786802, 0.3479212253829322, 0.3505747126436782, 0.3488372093023256, 0.34429065743944637, 0.34290540540540543, 0.3416666666666667, 0.34105960264900664, 0.3393739703459638, 0.34262295081967215, 0.3415032679738562, 0.34094616639477976, 0.34039087947882735, 0.34039087947882735, 0.33983739837398375, 0.33983739837398375, 0.33983739837398375, 0.33983739837398375, 0.33983739837398375], [0.31865284974093266, 0.32234432234432236, 0.3394216133942161, 0.3364485981308411, 0.3345724907063197, 0.32977461447212336, 0.32798165137614677, 0.32852386237513875, 0.3267543859649123, 0.3257328990228013, 0.3268608414239482, 0.32690246516613075, 0.32620320855614976, 0.3255069370330843, 0.326226012793177, 0.32553191489361705, 0.32553191489361705, 0.32553191489361705, 0.32553191489361705, 0.32553191489361705], [0.3081761006289308, 0.3069016152716593, 0.32406287787182586, 0.3227848101265823, 0.3213242453748783, 0.31691449814126393, 0.31687612208258525, 0.3165217391304348, 0.315068

In [58]:
#setup the 2D grid with Numpy
p_g, p_w = np.meshgrid(Pg_list, Pw_list)
        
#convert the result(percent) to a numpy array for plotting
P_list = np.array(P_list)

plt.pcolormesh(p_g, p_w, P_list)
plt.colorbar()
plt.savefig('C:/research/python/data/motif_enrichment.tif')